In [10]:
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv('swiggy_data.csv')  # Use raw string (r) for file path

# Step 2: Drop duplicate rows
df = df.drop_duplicates()

# Step 3: Drop rows with missing values
df = df.dropna()

# Step 4: Save the cleaned data (with index=False to avoid writing the index column)
df.to_csv('Cleaned_data.csv', index=False)  # Ensure file extension .csv and index=False
print("Data_Cleaned")


Data_Cleaned


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import pickle

# Load the cleaned data (cleaned_data.csv file)
df = pd.read_csv('cleaned_data.csv')

# Define the categorical columns to be encoded
categorical_columns = ['name', 'city', 'cuisine']

# Initialize the OneHotEncoder with sparse_output=True
encoder = OneHotEncoder(sparse_output=True)  # This returns a sparse matrix

# Apply One-Hot Encoding to the categorical columns
encoded_data = encoder.fit_transform(df[categorical_columns])

# Convert the sparse matrix to a DataFrame (if needed)
encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Drop the original categorical columns and concatenate the encoded columns
df_encoded = pd.concat([df.drop(columns=categorical_columns), encoded_df], axis=1)

# 2.1. Save the encoder as a Pickle file
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

# 2.2. Save the preprocessed dataset to a new CSV file
df_encoded.to_csv('encoded_data.csv', index=False)

# 2.3. Ensure the indices of cleaned_data.csv and encoded_data.csv match
assert df.index.equals(df_encoded.index), "Indices do not match! Ensure matching indices between cleaned_data and encoded_data."


In [3]:
import pandas as pd
import numpy as np

# Load encoded data safely
encoded_df = pd.read_csv('encoded_data.csv', low_memory=False)

# Replace invalid entries like '--', 'N/A', etc., with NaN
encoded_df.replace(['--', 'N/A', 'NaN', ''], np.nan, inplace=True)

# Drop or fill missing values (filling with 0 is safe for encoding)
encoded_df.fillna(0, inplace=True)

# Ensure all columns are numeric
encoded_df = encoded_df.apply(pd.to_numeric, errors='coerce').fillna(0)


In [4]:
# Check for any remaining non-numeric columns
non_numeric_cols = encoded_df.select_dtypes(exclude=[np.number]).columns
print("Non-numeric columns (should be empty):", non_numeric_cols)


Non-numeric columns (should be empty): Index([], dtype='object')


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

cleaned_df = pd.read_csv('cleaned_data.csv')

def recommend_restaurants(user_index, top_n=5):
    # Compute cosine similarity
    similarity_scores = cosine_similarity([encoded_df.iloc[user_index]], encoded_df)[0]
    
    # Get top similar indices (excluding itself)
    similar_indices = similarity_scores.argsort()[::-1][1:top_n+1]
    
    return cleaned_df.iloc[similar_indices]

# Example usage
recommendations = recommend_restaurants(user_index=10, top_n=5)
print(recommendations)


        id                       name    city rating     rating_count   cost  \
7   244866  Shri Balaji Vaishno Dhaba  Abohar     --  Too Few Ratings  ₹ 100   
24  435582  Picado International Food  Abohar     --  Too Few Ratings  ₹ 300   
23  427610                 Just Baked  Abohar     --  Too Few Ratings  ₹ 300   
2   158203          theka coffee desi  Abohar    3.8     100+ ratings  ₹ 100   
3   187912                  Singh Hut  Abohar    3.7      20+ ratings  ₹ 250   

             cuisine          lic_no  \
7       North Indian  22119652000389   
24  Pizzas,Beverages  22121652000286   
23  Beverages,Pizzas  22121652000339   
2          Beverages  22121652000190   
3   Fast Food,Indian  22119652000167   

                                                 link  \
7   https://www.swiggy.com/restaurants/shri-balaji...   
24  https://www.swiggy.com/restaurants/picado-inte...   
23  https://www.swiggy.com/restaurants/just-baked-...   
2   https://www.swiggy.com/restaurants/theka-coffe

In [6]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load data
cleaned_df = pd.read_csv('cleaned_data.csv')
encoded_df = pd.read_csv('encoded_data.csv')

# Load encoder
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# Clean encoded data to ensure all numerical
encoded_df.replace(['--', 'N/A', '', 'NaN'], np.nan, inplace=True)
encoded_df.fillna(0, inplace=True)
encoded_df = encoded_df.apply(pd.to_numeric, errors='coerce').fillna(0)

# Title
st.title("🍽️ Restaurant Recommendation System")

# --- User Input Section ---
st.sidebar.header("Filter Preferences")

# Select available options from cleaned data
cities = cleaned_df['city'].dropna().unique()
cuisines = cleaned_df['cuisine'].dropna().unique()
ratings = sorted(cleaned_df['rating'].dropna().unique())
prices = sorted(cleaned_df['price'].dropna().unique())

# Sidebar inputs
selected_city = st.sidebar.selectbox("Select City", cities)
selected_cuisine = st.sidebar.selectbox("Select Cuisine", cuisines)
min_rating = st.sidebar.slider("Minimum Rating", min(ratings), max(ratings), 3)
max_price = st.sidebar.slider("Maximum Price", min(prices), max(prices), max(prices))

# Filter original data
filtered_df = cleaned_df[
    (cleaned_df['city'] == selected_city) &
    (cleaned_df['cuisine'] == selected_cuisine) &
    (cleaned_df['rating'] >= min_rating) &
    (cleaned_df['price'] <= max_price)
]

# If no match
if filtered_df.empty:
    st.warning("No restaurants match your preferences.")
else:
    # Take first matching restaurant as input
    ref_index = filtered_df.index[0]

    # Compute similarity
    similarity_scores = cosine_similarity(
        [encoded_df.iloc[ref_index]], encoded_df
    )[0]

    # Get top 5 similar indices (excluding input)
    similar_indices = similarity_scores.argsort()[::-1][1:6]
    recommendations = cleaned_df.iloc[similar_indices]

    # Output section
    st.subheader("🍴 Top Recommended Restaurants")
    st.write(recommendations[['name', 'city', 'cuisine', 'rating', 'price']])



C:\Users\usen\AppData\Local\Temp\ipykernel_8760\4116975705.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  encoded_df = pd.read_csv('encoded_data.csv')
2025-04-06 11:20:01.507 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:20:05.618 
  command:

    streamlit run C:\Users\usen\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-06 11:20:05.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:20:05.626 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:20:05.630 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


KeyError: 'price'

In [9]:
st.write("🔍 Columns in cleaned_df:", cleaned_df.columns.tolist())


2025-04-06 11:22:29.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:22:29.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:22:29.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:22:29.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:22:29.088 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:22:29.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [10]:
# Try to detect a column related to price
price_col = next((col for col in cleaned_df.columns if 'price' in col.lower()), None)

if price_col:
    prices = sorted(cleaned_df[price_col].dropna().unique())
else:
    st.warning("⚠️ No 'price' column found. Please check the column names.")
    prices = []  # fallback so app won't crash


2025-04-06 11:22:52.942 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:22:52.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [11]:
cleaned_df.columns = cleaned_df.columns.str.strip().str.lower()


In [15]:
prices = sorted(cleaned_df['price'].dropna().unique())


KeyError: 'price'

In [13]:
print("COLUMNS:", cleaned_df.columns.tolist())


COLUMNS: ['id', 'name', 'city', 'rating', 'rating_count', 'cost', 'cuisine', 'lic_no', 'link', 'address', 'menu']


In [14]:
# Clean column names
cleaned_df.columns = cleaned_df.columns.str.strip().str.lower()

# Auto-detect any price-like column
price_col = next((col for col in cleaned_df.columns if 'price' in col), None)

if price_col:
    prices = sorted(cleaned_df[price_col].dropna().unique())
else:
    st.warning("⚠️ Couldn't find a 'price' column. Available columns: " + ", ".join(cleaned_df.columns))
    prices = []


2025-04-06 11:24:20.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 11:24:20.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
import streamlit as st
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# --- Load and preprocess data ---
@st.cache_data
def load_data():
    cleaned_df = pd.read_csv('cleaned_data.csv')
    encoded_df = pd.read_csv('encoded_data.csv', low_memory=False)

    # Normalize column names
    cleaned_df.columns = cleaned_df.columns.str.strip().str.lower()
    encoded_df.columns = encoded_df.columns.str.strip().str.lower()

    # Ensure 'rating' column is numeric
    if 'rating' in cleaned_df.columns:
        cleaned_df['rating'] = pd.to_numeric(cleaned_df['rating'], errors='coerce')

    return cleaned_df, encoded_df

cleaned_df, encoded_df = load_data()

# --- Auto-detect column names ---
city_col = next((col for col in cleaned_df.columns if 'city' in col), None)
cuisine_col = next((col for col in cleaned_df.columns if 'cuisine' in col), None)
rating_col = next((col for col in cleaned_df.columns if 'rating' in col), None)
price_col = next((col for col in cleaned_df.columns if 'price' in col), None)

# --- Sidebar user input ---
st.sidebar.title("🍽️ Restaurant Preferences")

if city_col and cuisine_col and rating_col:
    cities = sorted(cleaned_df[city_col].dropna().unique())
    cuisines = sorted(cleaned_df[cuisine_col].dropna().unique())
else:
    st.error("Missing expected columns in cleaned_data.csv")
    st.stop()

selected_city = st.sidebar.selectbox("City", cities)
selected_cuisine = st.sidebar.selectbox("Cuisine", cuisines)
selected_rating = st.sidebar.slider("Minimum Rating", min_value=0.0, max_value=5.0, step=0.5, value=3.5)

if price_col:
    try:
        prices = sorted(cleaned_df[price_col].dropna().unique())
        selected_price = st.sidebar.selectbox("Price Range", prices)
    except Exception as e:
        st.sidebar.warning(f"Price column error: {e}")
        selected_price = None
else:
    selected_price = None

# --- Filter matching restaurants ---
filtered_df = cleaned_df[
    (cleaned_df[city_col] == selected_city) &
    (cleaned_df[cuisine_col] == selected_cuisine) &
    (cleaned_df[rating_col] >= selected_rating)
]

if selected_price and price_col:
    filtered_df = filtered_df[filtered_df[price_col] == selected_price]

# --- Recommendation Engine ---
def recommend_restaurants(user_index, top_n=5):
    try:
        similarity_scores = cosine_similarity(
            [encoded_df.iloc[user_index]],
            encoded_df
        )[0]
        similar_indices = similarity_scores.argsort()[::-1][1:top_n+1]
        return cleaned_df.iloc[similar_indices]
    except Exception as e:
        st.error(f"Recommendation error: {e}")
        return pd.DataFrame()

# --- Output ---
st.title("🍴 Restaurant Recommender")

if not filtered_df.empty:
    st.success(f"Found {len(filtered_df)} matching restaurants.")
    
    # Pick the first matching restaurant as reference
    user_index = filtered_df.index[0]
    recommendations = recommend_restaurants(user_index=user_index, top_n=5)
    
    st.subheader("Top Recommendations 🍕")
    st.dataframe(recommendations.reset_index(drop=True))
else:
    st.warning("No matching restaurants found. Try adjusting your filters.")


2025-04-06 15:47:23.557 No runtime found, using MemoryCacheStorageManager
2025-04-06 15:47:23.579 No runtime found, using MemoryCacheStorageManager
2025-04-06 15:47:23.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:47:23.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:47:23.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:47:24.610 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:47:24.636 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:47:42.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:47:42.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored w

In [3]:
import pandas as pd

df = pd.read_csv('cleaned_data.csv')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')  # Convert to float
filtered_df = df[df['rating'] >= 4.0]


In [4]:
cleaned_df['rating'] = pd.to_numeric(cleaned_df['rating'], errors='coerce')
filtered_df = cleaned_df[cleaned_df['rating'] >= 4.0]


In [5]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df = df[df['rating'] >= 4.0]  # Now this will work


In [6]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')


KeyError: 'price'